In [ ]:
%%bash 
cd ~/SageMaker
mkdir input 
mkdir input/data 
cd input/data 
wget -O coco.zip https://tinyurl.com/yhtbxr6q 
unzip coco.zip 

In [2]:
%%bash 
./build-and-push.sh 

230755935769
us-west-2
Login Succeeded
Login Succeeded
base_img:763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04

Step 1/12 : ARG BASE_IMG=${BASE_IMG}
Step 2/12 : FROM ${BASE_IMG}
 ---> 026ade4c903c
Step 3/12 : ENV PATH="/opt/ml/code:/usr/local/cuda/bin/:${PATH}"
 ---> Using cache
 ---> bd4e8cc3a612
Step 4/12 : ENV LD_LIBRARY_PATH="/usr/local/cuda/lib64/:$LD_LIBRARY_PATH"
 ---> Using cache
 ---> bb62cc14adaa
Step 5/12 : RUN python -m pip install cython
 ---> Using cache
 ---> 71e3419800eb
Step 6/12 : RUN apt-get update && apt-get install libyaml-dev locales -y
 ---> Using cache
 ---> 02bfc3bf5f4d
Step 7/12 : RUN export LANG=C.UTF-8
 ---> Using cache
 ---> 81e90f39c6d3
Step 8/12 : RUN git clone https://github.com/catwhiskers/AlphaPose.git  && cd AlphaPose && python setup.py build develop
 ---> Running in 0b29af4c02f1
Cloning into 'AlphaPose'...
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running build
running build_py
creatin

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

--2021-03-14 10:58:19--  https://tinyurl.com/yzaqy5db
Resolving tinyurl.com (tinyurl.com)... 104.20.138.65, 104.20.139.65, 172.67.1.225, ...
Connecting to tinyurl.com (tinyurl.com)|104.20.138.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://alphapose-yianc.s3.amazonaws.com/assets/yolov3-spp.weights?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential

In [5]:
!nvidia-docker run -it -v /home/ec2-user/SageMaker/:/opt/ml 230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos "cd /AlphaPose/ \
&& python scripts/demo_inference.py \
--cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml \
--checkpoint pretrained_models/fast_res50_256x192.pth \
--indir examples/demo/ \
--outdir /opt/ml/demo --save_img"

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|███████████████████████████████████████| 97.8M/97.8M [00:00<00:00, 154MB/s]
Loading YOLO model..
Loading pose model from pretrained_models/fast_res50_256x192.pth...
  0%|                                                     | 0/3 [00:00<?, ?it/s][2021-03-14 11:08:41.624 28d09da4ea1d:26 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-03-14 11:08:41.951 28d09da4ea1d:26 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
/AlphaPose/detector/yolo_api.py:133: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /codebuild/output/src811146734/src/torch/csrc/utils/python_arg_parser.cpp:766.)
  ind_nz = torch.nonzero(

In [7]:
!nvidia-docker run -it -v /home/ec2-user/SageMaker/:/opt/ml 230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos "cd /AlphaPose/ \
&& ./scripts/train.sh \
./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml \
exp_fastpose"

+++ CONFIG=./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml
+++ EXPID=exp_fastpose
+++ python ./scripts/train.py --exp-id exp_fastpose --cfg ./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --nThreads 1
Generating new fontManager, this may take some time...
num_gpu 1
******************************
Namespace(board=True, cfg='./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml', debug=False, detector='yolo', device=device(type='cuda', index=0), dist_backend='nccl', dist_url='tcp://192.168.1.214:23345', exp_id='exp_fastpose', gpus=[0], launcher='none', map=True, nThreads=1, rank=-1, snapshot=2, sync=False, work_dir='./exp/exp_fastpose-256x192_res50_lr1e-3_1x.yaml/', world_size=1)
******************************
{'DATASET': {'TRAIN': {'TYPE': 'Mscoco', 'ROOT': '/opt/ml/input/data/coco/', 'IMG_PREFIX': 'val2017', 'ANN': 'annotations/person_keypoints_val2017.json', 'AUG': {'FLIP': True, 'ROT_FACTOR': 40, 'SCALE_FACTOR': 0.3, 'NUM_JOINTS_HALF_BODY': 8, 'PROB_HALF_BODY': -1}}, 'VAL': {'TYPE'

In [19]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "alphapose"

In [20]:
!cd ~/SageMaker/input/data/ && aws s3 cp --recursive coco s3://{bucket}/{prefix}/coco

upload: coco/annotations/captions_val2017.json to s3://sagemaker-us-west-2-230755935769/alphapose/coco/annotations/captions_val2017.json
upload: coco/annotations/instances_val2017.json to s3://sagemaker-us-west-2-230755935769/alphapose/coco/annotations/instances_val2017.json
upload: coco/annotations/person_keypoints_val2017.json to s3://sagemaker-us-west-2-230755935769/alphapose/coco/annotations/person_keypoints_val2017.json
upload: coco/annotations/captions_train2017.json to s3://sagemaker-us-west-2-230755935769/alphapose/coco/annotations/captions_train2017.json
upload: coco/annotations/person_keypoints_train2017.json to s3://sagemaker-us-west-2-230755935769/alphapose/coco/annotations/person_keypoints_train2017.json
upload: coco/val2017/000000000139.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000000139.jpg
upload: coco/val2017/000000000285.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000000285.jpg
upload: coco/val2017/000000000632.j

upload: coco/val2017/000000006723.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000006723.jpg
upload: coco/val2017/000000006894.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000006894.jpg
upload: coco/val2017/000000006954.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000006954.jpg
upload: coco/val2017/000000007281.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000007281.jpg
upload: coco/val2017/000000007088.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000007088.jpg
upload: coco/val2017/000000007278.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000007278.jpg
upload: coco/val2017/000000007386.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000007386.jpg
upload: coco/val2017/000000007511.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000007511.jpg
upload: coco/val2017/000000005586.jpg to s3://sa

upload: coco/val2017/000000014007.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014007.jpg
upload: coco/val2017/000000014038.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014038.jpg
upload: coco/val2017/000000014226.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014226.jpg
upload: coco/val2017/000000014473.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014473.jpg
upload: coco/val2017/000000014439.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014439.jpg
upload: coco/val2017/000000014380.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014380.jpg
upload: coco/val2017/000000013348.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000013348.jpg
upload: coco/val2017/000000014888.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000014888.jpg
upload: coco/val2017/000000014831.jpg to s3://sa

upload: coco/val2017/000000021503.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021503.jpg
upload: coco/val2017/000000021604.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021604.jpg
upload: coco/val2017/000000020571.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000020571.jpg
upload: coco/val2017/000000021839.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021839.jpg
upload: coco/val2017/000000021167.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021167.jpg
upload: coco/val2017/000000021465.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021465.jpg
upload: coco/val2017/000000021903.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021903.jpg
upload: coco/val2017/000000021879.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000021879.jpg
upload: coco/val2017/000000022623.jpg to s3://sa

upload: coco/val2017/000000029187.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000029187.jpg
upload: coco/val2017/000000029675.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000029675.jpg
upload: coco/val2017/000000029397.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000029397.jpg
upload: coco/val2017/000000030213.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000030213.jpg
upload: coco/val2017/000000029393.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000029393.jpg
upload: coco/val2017/000000030494.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000030494.jpg
upload: coco/val2017/000000030504.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000030504.jpg
upload: coco/val2017/000000030828.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000030828.jpg
upload: coco/val2017/000000030675.jpg to s3://sa

...

upload: coco/val2017/000000548339.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000548339.jpg
upload: coco/val2017/000000548780.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000548780.jpg
upload: coco/val2017/000000548246.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000548246.jpg
upload: coco/val2017/000000549220.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000549220.jpg
upload: coco/val2017/000000549055.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000549055.jpg
upload: coco/val2017/000000549930.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000549930.jpg
upload: coco/val2017/000000549738.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000549738.jpg
upload: coco/val2017/000000549390.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000549390.jpg
upload: coco/val2017/000000549167.jpg to s3://sa

upload: coco/val2017/000000558213.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000558213.jpg
upload: coco/val2017/000000558558.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000558558.jpg
upload: coco/val2017/000000558854.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000558854.jpg
upload: coco/val2017/000000555050.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000555050.jpg
upload: coco/val2017/000000558073.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000558073.jpg
upload: coco/val2017/000000557672.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000557672.jpg
upload: coco/val2017/000000559348.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000559348.jpg
upload: coco/val2017/000000558114.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000558114.jpg
upload: coco/val2017/000000558421.jpg to s3://sa

upload: coco/val2017/000000565778.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565778.jpg
upload: coco/val2017/000000565563.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565563.jpg
upload: coco/val2017/000000565597.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565597.jpg
upload: coco/val2017/000000565853.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565853.jpg
upload: coco/val2017/000000565877.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565877.jpg
upload: coco/val2017/000000565012.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565012.jpg
upload: coco/val2017/000000565962.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565962.jpg
upload: coco/val2017/000000565469.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000565469.jpg
upload: coco/val2017/000000565989.jpg to s3://sa

upload: coco/val2017/000000573094.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000573094.jpg
upload: coco/val2017/000000573626.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000573626.jpg
upload: coco/val2017/000000574315.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000574315.jpg
upload: coco/val2017/000000574297.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000574297.jpg
upload: coco/val2017/000000573943.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000573943.jpg
upload: coco/val2017/000000573258.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000573258.jpg
upload: coco/val2017/000000573391.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000573391.jpg
upload: coco/val2017/000000574520.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000574520.jpg
upload: coco/val2017/000000574702.jpg to s3://sa

upload: coco/val2017/000000581482.jpg to s3://sagemaker-us-west-2-230755935769/alphapose/coco/val2017/000000581482.jpg


In [ ]:
coco_data = "s3://{}/{}/coco/".format(bucket, prefix)
outpath = "s3://{}/{}/output/".format(bucket, prefix)
repositoryUri = "230755935769.dkr.ecr.us-west-2.amazonaws.com/alphapose-byos"

In [13]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "alphapose-{}".format(str(int(timestamp))) 
job_name 

'alphapose-1615720783'

In [14]:
hyperparameters = {
 "exp-id": "fastpose",
 "cfg": "/AlphaPose/configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml",
 "nThreads": 1
 }

In [15]:
coco_input = sagemaker.inputs.TrainingInput(coco_data)

In [16]:
!cp scripts/train.py . 

In [17]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                        role=role,
                        image_uri=repositoryUri,
                        source_dir='.',
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        framework_version='1.6.0',
                        py_version='py3',
                        sagemaker_session=sagemaker_session,
                        hyperparameters=hyperparameters, 
                        volume_size=100, 
                        debugger_hook_config=False,
                        # Training using SMDataParallel Distributed Training Framework
#                         distribution={'smdistributed':{
#                                             'dataparallel':{
#                                                     'enabled': True
#                                                  }
#                                           }
#                                       }
                   )


In [18]:
estimator.fit(inputs={"coco":coco_input}, job_name=job_name)

2021-03-14 11:24:24 Starting - Starting the training job...
2021-03-14 11:24:48 Starting - Launching requested ML instancesProfilerReport-1615720787: InProgress
.........
2021-03-14 11:26:08 Starting - Preparing the instances for training......
2021-03-14 11:27:08 Downloading - Downloading input data......
2021-03-14 11:28:10 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-03-14 11:32:27,821 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-03-14 11:32:27,863 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-03-14 11:32:30,892 sagemaker_pytorch_container.training INFO     Invoking user training script.

2021-03-14 11:33:11 Training - Training image download completed. Training in progress.2021-03-14 11:33:36,175 sagemaker-training-toolkit INFO     Installing module

loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
[2021-03-14 11:34:38.944 algo-1:204 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-03-14 11:34:39.170 algo-1:204 INFO profiler_config_parser.py:102] User has disabled profiler.
NCCL version 2.4.8+cuda11.0
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Generating new fontManager, this may take some time...
******************************
Namespace(board=True, cfg='/AlphaPose/configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml', debug=False, detector='yolo', device=device(type='cuda', index=0), dist_backend='nccl', dist_url='tcp://192.168.1.214:23345', exp_id='fastpose', gpus=[0, 1, 2, 3], launcher='none', map=True, nThreads=1, rank=-1, snapshot=2, sync=False, work_dir='./exp/fastpose-256x192_res50_lr1e-3_1x.yaml/', world_size=1)
******************************
{'DATASET': {'TRAIN': {'TYPE': 'Mscoco', 'ROOT': '/opt/ml/input/data/coco/', 'IMG_PREFIX': 'val2017', '


2021-03-14 11:47:33 Uploading - Uploading generated training model
2021-03-14 11:47:33 Completed - Training job completed
ProfilerReport-1615720787: IssuesFound
Training seconds: 1204
Billable seconds: 1204
